In [1]:
!pip install -q transformers rich torchmetrics sklearn pytorch-lightning sentence-transformers

In [2]:
import pandas as pd
import numpy as np
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3
import dask.bag as db
import json
import zipfile
from sklearn.preprocessing import MultiLabelBinarizer
import csv
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig
from sklearn.model_selection import train_test_split
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import RichProgressBar
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from torchmetrics import F1
from torchmetrics.functional import accuracy
from transformers import RobertaTokenizer, RobertaModel, AutoTokenizer, AutoModel
from transformers import AutoModel, AdamW, get_cosine_schedule_with_warmup
import math
from torchmetrics.functional.classification import auroc
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer
import pickle
from rich.progress import track
import pytorch_lightning as pl



In [3]:
!mkdir dataset

mkdir: cannot create directory ‘dataset’: File exists


In [4]:
raw_json_data = "arxiv-metadata-oai-snapshot-json.zip"
pickled_dataset = 'dataset_pickle.pkl'
pickled_dataset_onehot = 'dataset_multilabel.pkl'

In [5]:
def __iter__(self): return 0

client_c2b4ce19d76d4b7c87bdf6c8c84d662c = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='SECRET',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

streaming_body_1 = client_c2b4ce19d76d4b7c87bdf6c8c84d662c.get_object(Bucket='SECRET', Key='arxiv-metadata-oai-snapshot.json.zip')['Body']

def save_to_cloud(key, file):
    client_c2b4ce19d76d4b7c87bdf6c8c84d662c.upload_file(Bucket='SECRET', Key=f'snapshot-{file}', Filename=file)

    
def get_file(key, filename):
    client_c2b4ce19d76d4b7c87bdf6c8c84d662c.download_file(Bucket='SECRET', Key=key, Filename=filename)

In [6]:
get_file('snapshot-uni_df.pkl', 'uni_df.pkl')

df_onehot = pd.read_pickle('uni_df.pkl')
LABEL_COLUMNS = df_onehot.columns.tolist()[1:]

In [7]:
class ArxivAbstracts(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.text = df.abstract
        self.labels = df[df.columns.difference(['abstract'])]
        self.max_len = max_len
        
    def __len__(self):
        return len(self.text)
    
    def __getitem__(self, item_idx):
        text = self.text[item_idx]
        encoding = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            return_token_type_ids=False,
            return_attention_mask= True,
            truncation=True,
            return_tensors='pt'
          )
               
        return {
          'ids': encoding["input_ids"].flatten(),
          'mask': encoding["attention_mask"].flatten(),
          'targets': torch.tensor(self.labels.iloc[item_idx].tolist(), dtype=torch.float)  
        }

In [8]:
RANDOM_SEED = 1337
df_train, df_test = train_test_split(df_onehot, test_size=0.1, random_state=RANDOM_SEED, shuffle=True)
df_train, df_validation = train_test_split(df_train, test_size=0.2, random_state=RANDOM_SEED, shuffle=True)
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_validation = df_validation.reset_index(drop=True)

In [9]:
import pytorch_lightning as pl
class ArxivAbstractsModule(pl.LightningDataModule):
    
    def __init__(self, df_train, df_test, df_validation, tokenizer, batch_size=16, max_token_len=256):
        super().__init__()
        self.df_train = df_train
        self.df_test = df_test
        self.df_validation = df_validation
        self.tokenizer = tokenizer
        self.batch_size = batch_size
        self.max_token_len = max_token_len

    def setup(self, stage=None):
        self.train_dataset = ArxivAbstracts(df=self.df_train, tokenizer=self.tokenizer, max_len= self.max_token_len)
        self.val_dataset = ArxivAbstracts(df=self.df_validation, tokenizer=self.tokenizer, max_len= self.max_token_len)
        self.test_dataset = ArxivAbstracts(df=self.df_test, tokenizer=self.tokenizer, max_len= self.max_token_len)


    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size)

In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
arxiv_data = ArxivAbstractsModule(df_train, df_test, df_validation, tokenizer, max_token_len=384)
arxiv_data.setup()

In [11]:
class ArxivAbstractsClassifier(pl.LightningModule):
    def __init__(self, n_classes, lr=2e-5):
        super().__init__()

        self.bert = BertModel.from_pretrained("bert-base-cased", return_dict=True)
        self.classifier = torch.nn.Linear(self.bert.config.hidden_size, n_classes) 
        self.lr = lr
        self.criterion = torch.nn.BCEWithLogitsLoss()
        
        self.f1_metrics = torch.nn.ModuleDict({
            'f1_weighted': F1(num_classes=len(LABEL_COLUMNS), average='weighted', threshold=0.5),       
            'f1_samples': F1(num_classes=len(LABEL_COLUMNS), average='samples', threshold=0.5), 
            'f1_micro': F1(num_classes=len(LABEL_COLUMNS), average='micro', threshold=0.5),
            'f1_macro': F1(num_classes=len(LABEL_COLUMNS), average='macro', threshold=0.5),
            'f1_weighted_t02': F1(num_classes=len(LABEL_COLUMNS), average='weighted', threshold=0.2),       
            'f1_samples_t02': F1(num_classes=len(LABEL_COLUMNS), average='samples', threshold=0.2), 
            'f1_micro_t02': F1(num_classes=len(LABEL_COLUMNS), average='micro', threshold=0.2),
            'f1_macro_t02': F1(num_classes=len(LABEL_COLUMNS), average='macro', threshold=0.2),
        })
        self.save_hyperparameters()

    def compute_metrics(self, outputs, labels):
        metrics = {}
        for name, metric in self.f1_metrics.items():
            metrics[name] = metric(outputs, labels)
        return metrics  
    
    def forward(self, ids=None, mask=None):
        print(ids)
        print(mask)
#         if ids is None:
#             ids = batch[0]
#         if mask is None:
#             mask = batch[1]
        
        output = self.bert(input_ids=ids, attention_mask=mask)
        output = self.classifier(output.pooler_output)   
        return output
    
    def loss(self, batch, prediction):
        return self.criterion(prediction, batch["targets"])
    
    def __generic_step(self, batch, batch_idx, namespace: str):
        input_ids = batch["ids"]
        attention_mask = batch["mask"]
        
        outputs = self(input_ids, attention_mask)
        predictions = torch.sigmoid(outputs)
        loss = self.loss(batch, outputs)
        
        if namespace != "pred":
            labels = batch["targets"]
            metrics = self.compute_metrics(predictions, labels.int())
            self.log_dict({f'{namespace}_loss': loss, **
                           {f'{namespace}_{k}': v for k, v in metrics.items()}})
        
        return {"loss": loss, "predictions": predictions}
        
    def predict_step(self, batch, batch_idx):
        return self.__generic_step(batch, batch_idx, namespace="pred")["predictions"]
    
    def training_step(self, batch, batch_idx):
        return self.__generic_step(batch, batch_idx, namespace="train")["loss"]
        return loss

    def validation_step(self, batch, batch_idx):
        return self.__generic_step(batch, batch_idx, namespace="val")["loss"]
    
    def test_step(self, batch, batch_idx):
        return self.__generic_step(batch, batch_idx, namespace="test")["loss"]

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(params=self.parameters(), lr=self.lr)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)
        return {
            'optimizer': optimizer,
            'lr_scheduler': scheduler,
            'monitor': 'val_loss'
        }


In [12]:
model_name = "ArxivAbstractsClassifierPreprocessData5Epoch"

In [13]:

logger = TensorBoardLogger("lightning_logs", name="arxiv-abstracts-logs")

In [14]:
get_file(f'snapshot-{model_name}.ckpt', f'snapshot-{model_name}.ckpt')

In [15]:
wml_credentials = {
    'apikey': 'SECRET',
    "url"         : "https://eu-gb.ml.cloud.ibm.com",
}

space_id = "6f48a9aa-b319-450c-b6f1-bd3a789704d4"


In [16]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)

In [17]:
client.spaces.list(limit=10)


------------------------------------  ------------  ------------------------
ID                                    NAME          CREATED
6f48a9aa-b319-450c-b6f1-bd3a789704d4  arxiv-papers  2022-06-11T20:25:39.649Z
------------------------------------  ------------  ------------------------


In [18]:
client.set.default_space(space_id)


'SUCCESS'

In [19]:
model = ArxivAbstractsClassifier(n_classes=151).load_from_checkpoint(f"snapshot-{model_name}.ckpt", n_classes=151)
model.eval()

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transf

ArxivAbstractsClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [20]:
input_sample = None
for d in arxiv_data.val_dataloader():
    input_sample = d
    break

In [21]:
input_sample

{'ids': tensor([[  101,  2508,  2225,  ...,     0,     0,     0],
         [  101,  1130,  1251,  ...,     0,     0,     0],
         [  101,  1188,  2526,  ...,     0,     0,     0],
         ...,
         [  101,   142,  2050,  ...,     0,     0,     0],
         [  101, 11336, 16236,  ..., 23896,  7008,   102],
         [  101,  5732, 10393,  ...,     0,     0,     0]]),
 'mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'targets': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]])}

In [341]:
# sampled = (input_sample["ids"], input_sample)

In [342]:
sampled

(tensor([[  101,  2508,  2225,  6660,  1103,  9539,  8096,   113,   154,  2036,
            114,  4520,  1104,   170,  6307, 12650,  5114,  2007,  1107, 16811,
           1766,  3417,  5315, 10221, 15049,   117,  1126, 19849,  2805,  1115,
           6808, 22015,  1158,  1103,  1107, 16811,  1766, 10221,  2817,  1105,
            187, 20517,  1158,  1103,  3378, 10221,  3205,  1506,  1103,  6307,
          12650,  5114,  2007,  2473,   119, 16890,  4648, 15049, 19717,  1116,
           2999,  2805,  1105,  2274,  5602,  1159,  1106, 18011,   119,  1130,
           1142,  2526,   117,  1195, 10541,   170,  2281,  3442,  1104, 10099,
           1103,   154,  2036,  4520,  1606,   170,  7483, 14377,  8297,  1115,
           1884, 18337,  5430,  1103,  1107, 16811,  1766, 10221, 15442,  8516,
           1166,  1159,  1114,  1103,  1703,  2965, 10972,   119,  9040, 14377,
          13267, 14403,   117,  1842,   118,  1159,  9437,  1104,  1103,   154,
           2036,  4520,  1443, 23465, 22

In [22]:
filename = 'torch_mlp.onnx'
tar_filename = filename + '.tgz'

torch.onnx.export(
        model,
        args=tuple([input_sample["ids"], input_sample["mask"]]),
        f=filename,
        input_names=['ids', 'mask'],
        do_constant_folding=False,
        use_external_data_format=False,
        enable_onnx_checker=True,
        opset_version=12,
    )

/opt/conda/envs/Python-3.9-CUDA/lib/python3.9/site-packages/torch/onnx/utils.py:90: UserWarning: 'enable_onnx_checker' is deprecated and ignored. It will be removed in the next PyTorch release. To proceed despite ONNX checker failures, catch torch.onnx.ONNXCheckerError.
  warnings.warn("'enable_onnx_checker' is deprecated and ignored. It will be removed in "
/opt/conda/envs/Python-3.9-CUDA/lib/python3.9/site-packages/torch/onnx/utils.py:103: UserWarning: `use_external_data_format' is deprecated and ignored. Will be removed in next PyTorch release. The code will work as it is False if models are not larger than 2GB, Otherwise set to False because of size limits imposed by Protocol Buffers.
  warnings.warn("`use_external_data_format' is deprecated and ignored. Will be removed in next "


tensor([[  101,  2508,  2225,  ...,     0,     0,     0],
        [  101,  1130,  1251,  ...,     0,     0,     0],
        [  101,  1188,  2526,  ...,     0,     0,     0],
        ...,
        [  101,   142,  2050,  ...,     0,     0,     0],
        [  101, 11336, 16236,  ..., 23896,  7008,   102],
        [  101,  5732, 10393,  ...,     0,     0,     0]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]])


/opt/conda/envs/Python-3.9-CUDA/lib/python3.9/site-packages/torch/onnx/symbolic_helper.py:325: UserWarning: Type cannot be inferred, which might cause exported graph to produce incorrect results.
  warnings.warn("Type cannot be inferred, which might cause exported graph to produce incorrect results.")
[W shape_type_inference.cpp:434] Warning: Constant folding in symbolic shape inference fails: index_select(): Index is supposed to be a vector
Exception raised from index_select_out_cpu_ at /opt/conda/conda-bld/pytorch-base_1645116610908/work/aten/src/ATen/native/TensorAdvancedIndexing.cpp:887 (most recent call first):
frame #0: c10::Error::Error(c10::SourceLocation, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >) + 0x6a (0x7f4eea37c98a in /opt/conda/envs/Python-3.9-CUDA/lib/python3.9/site-packages/torch/lib/libc10.so)
frame #1: at::native::index_select_out_cpu_(at::Tensor const&, long, at::Tensor const&, at::Tensor&) + 0x3d0 (0x7f4f19adbd60 in /opt/conda/

In [23]:
cmdstring = 'tar -zcvf ' + tar_filename + ' ' + filename
os.system(cmdstring)

torch_mlp.onnx


0

In [135]:
!ls

dataset							    torch_mlp.onnx
deployment.onnx						    torch_mlp.onnx.tgz
snapshot-ArxivAbstractsClassifierPreprocessData5Epoch.ckpt  uni_df.pkl


In [216]:
sofware_spec_uid = client.software_specifications.get_id_by_name("pytorch-onnx_rt22.1-py3.9")


In [24]:
metadata = {
            client.repository.ModelMetaNames.NAME: 'Classifier deployment',
            client.repository.ModelMetaNames.TYPE: 'pytorch-onnx_1.10',
            client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid,
}


NameError: name 'sofware_spec_uid' is not defined

In [297]:
published_model = client.repository.store_model(
    model=tar_filename,
    meta_props=metadata)

Failure during creating model. (POST https://eu-gb.ml.cloud.ibm.com/ml/v4/models?version=2021-06-24&space_id=6f48a9aa-b319-450c-b6f1-bd3a789704d4)
Status code: 400, body: {
  "trace": "24550975c4418f6d37bb926da522a5b6",
  "errors": [{
    "code": "malformed_json",
    "message": "Malformed create model json payload: Object is missing required member 'fields'",
    "target": {
      "type": "field",
      "name": "fields"
    },
    "more_info": "https://cloud.ibm.com/apidocs/machine-learning"
  }],
  "status_code": "400"
}


ApiRequestFailure: Failure during creating model. (POST https://eu-gb.ml.cloud.ibm.com/ml/v4/models?version=2021-06-24&space_id=6f48a9aa-b319-450c-b6f1-bd3a789704d4)
Status code: 400, body: {
  "trace": "24550975c4418f6d37bb926da522a5b6",
  "errors": [{
    "code": "malformed_json",
    "message": "Malformed create model json payload: Object is missing required member 'fields'",
    "target": {
      "type": "field",
      "name": "fields"
    },
    "more_info": "https://cloud.ibm.com/apidocs/machine-learning"
  }],
  "status_code": "400"
}

In [219]:
import json

published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

This method is deprecated, please use get_model_id()


/opt/conda/envs/Python-3.9-CUDA/lib/python3.9/site-packages/ibm_watson_machine_learning/repository.py:1452: UserWarning: This method is deprecated, please use get_model_id()
  warn("This method is deprecated, please use get_model_id()")


{
  "entity": {
    "hybrid_pipeline_software_specs": [],
    "software_spec": {
      "id": "0b848dd4-e681-5599-be41-b5f6fccc6471",
      "name": "pytorch-onnx_rt22.1-py3.9"
    },
    "type": "pytorch-onnx_1.10"
  },
  "metadata": {
    "created_at": "2022-06-12T13:02:46.447Z",
    "id": "2c90decd-0dc0-41f7-bbca-e3952282785e",
    "modified_at": "2022-06-12T13:03:23.298Z",
    "name": "Classifier deployment",
    "owner": "IBMid-6650023OBW",
    "resource_key": "6a6047bf-b4bf-4097-8a8c-83250b35ca59",
    "space_id": "6f48a9aa-b319-450c-b6f1-bd3a789704d4"
  },
  "system": {
    "warnings": []
  }
}


In [220]:
published_model_uid

'2c90decd-0dc0-41f7-bbca-e3952282785e'

In [221]:
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Deployment of external pytorch model",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

created_deployment = client.deployments.create(published_model_uid, meta_props=metadata)



#######################################################################################

Synchronous deployment creation for uid: '2c90decd-0dc0-41f7-bbca-e3952282785e' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
...
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='1e3114a8-13ff-4ed1-b2f1-eb731af71e95'
------------------------------------------------------------------------------------------------




In [28]:
deployment_id = "1e3114a8-13ff-4ed1-b2f1-eb731af71e95"

In [42]:
inps = pd.DataFrame([{"ids": input_sample["ids"].tolist(), "mask": input_sample["mask"].tolist()}])

In [62]:
fields = ["ids", "mask"]
values = [[input_sample["ids"].numpy()[0], input_sample["mask"].numpy()[0]]]

scoring_payload = {
client.deployments.ScoringMetaNames.INPUT_DATA: [{  
    "values": [
        {"ids": input_sample["ids"].tolist(), "mask": input_sample["mask"].tolist()}
    ]
 }]
}

In [65]:
client.repository.list_models()

------------------------------------  ---------------------  ------------------------  -----------------
ID                                    NAME                   CREATED                   TYPE
2c90decd-0dc0-41f7-bbca-e3952282785e  Classifier deployment  2022-06-12T13:02:46.002Z  pytorch-onnx_1.10
075da344-3252-4b3c-9dc6-c8c190df1434  Classifier deployment  2022-06-12T12:51:07.002Z  pytorch-onnx_1.10
0856adc0-edb9-4221-863e-1028b302802a  Classifier deployment  2022-06-12T11:54:13.002Z  pytorch-onnx_1.10
1467e23d-ceb2-4800-8b53-5a8fa62a2e60  Classifier deployment  2022-06-12T11:53:31.002Z  pytorch-onnx_1.7
------------------------------------  ---------------------  ------------------------  -----------------


In [87]:
client.deployments.get_scoring_url(deployment_id)

AttributeError: 'Deployments' object has no attribute 'get_scoring_url'

In [67]:
predictions = client.deployments.score(deployment_id, scoring_payload)


Failure during scoring. (POST https://eu-gb.ml.cloud.ibm.com/ml/v4/deployments/1e3114a8-13ff-4ed1-b2f1-eb731af71e95/predictions?version=2021-06-24)
Status code: 400, body: {"trace": "210a67ca7df30f943a55d566286ad1d3", "errors": [{"code": "invalid_input_data", "message": "Incorrect input data: int() argument must be a string, a bytes-like object or a number, not 'dict'"}], "status_code": 400}



ApiRequestFailure: Failure during scoring. (POST https://eu-gb.ml.cloud.ibm.com/ml/v4/deployments/1e3114a8-13ff-4ed1-b2f1-eb731af71e95/predictions?version=2021-06-24)
Status code: 400, body: {"trace": "210a67ca7df30f943a55d566286ad1d3", "errors": [{"code": "invalid_input_data", "message": "Incorrect input data: int() argument must be a string, a bytes-like object or a number, not 'dict'"}], "status_code": 400}


In [33]:
import onnx

model = onnx.load(filename)


In [34]:
onnx.checker.check_model(model)

In [35]:
model.graph.input

[name: "ids"
type {
  tensor_type {
    elem_type: 7
    shape {
      dim {
        dim_value: 16
      }
      dim {
        dim_value: 384
      }
    }
  }
}
, name: "mask"
type {
  tensor_type {
    elem_type: 7
    shape {
      dim {
        dim_value: 16
      }
      dim {
        dim_value: 384
      }
    }
  }
}
]

In [36]:
print(onnx.helper.printable_graph(model.graph))


graph torch-jit-export (
  %ids[INT64, 16x384]
  %mask[INT64, 16x384]
) initializers (
  %bert.embeddings.position_ids[INT64, 1x512]
  %bert.embeddings.word_embeddings.weight[FLOAT, 28996x768]
  %bert.embeddings.position_embeddings.weight[FLOAT, 512x768]
  %bert.embeddings.token_type_embeddings.weight[FLOAT, 2x768]
  %bert.embeddings.LayerNorm.weight[FLOAT, 768]
  %bert.embeddings.LayerNorm.bias[FLOAT, 768]
  %bert.encoder.layer.0.attention.self.query.weight[FLOAT, 768x768]
  %bert.encoder.layer.0.attention.self.query.bias[FLOAT, 768]
  %bert.encoder.layer.0.attention.self.key.weight[FLOAT, 768x768]
  %bert.encoder.layer.0.attention.self.key.bias[FLOAT, 768]
  %bert.encoder.layer.0.attention.self.value.weight[FLOAT, 768x768]
  %bert.encoder.layer.0.attention.self.value.bias[FLOAT, 768]
  %bert.encoder.layer.0.attention.output.dense.weight[FLOAT, 768x768]
  %bert.encoder.layer.0.attention.output.dense.bias[FLOAT, 768]
  %bert.encoder.layer.0.attention.output.LayerNorm.weight[FLOAT, 768]

In [37]:
!pip install onnxruntime

In [38]:
import onnxruntime as ort

ort_session = ort.InferenceSession(filename)


In [39]:
input_name = ort_session.get_inputs()[0].name
output_name = ort_session.get_outputs()[0].name

In [40]:
input_name

'ids'

In [41]:

# tuple([input_sample["ids"].tolist(), input_sample["mask"].tolist()])

ort_inputs = {
    'ids':  input_sample["ids"].tolist(),
    'mask': input_sample["mask"].tolist(),
}

outputs = ort_session.run(None, ort_inputs)

In [213]:
preds = torch.sigmoid(torch.FloatTensor(outputs[0]))

In [214]:
def predict(tresh = 0.5):
    y_pred = [[int(val >= tresh) for val in pred] for pred in preds]
    y_true = input_sample["targets"].tolist()
#     print(y_pred)
#     print(y_true)
    
    print(classification_report(y_true, y_pred))
    print("Acc score: ", accuracy_score(y_true, y_pred))

In [215]:
predict()

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       1.00      0.50      0.67         2
           4       1.00      1.00      1.00         1
           5       0.00      0.00      0.00         0
           6       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         1
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         1
          10       1.00      1.00      1.00         1
          11       1.00      1.00      1.00         1
          12       1.00      1.00      1.00         1
          13       0.00      0.00      0.00         0
          14       0.00      0.00      0.00         0
          15       0.00      0.00      0.00         0
          16       0.00      0.00      0.00         0
          17       0.00    

/opt/conda/envs/Python-3.9-CUDA/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/Python-3.9-CUDA/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/Python-3.9-CUDA/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
